Follow along: https://python.langchain.com/docs/use_cases/graph/constructing/

In [1]:
import os
import time

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_fireworks import ChatFireworks
from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# initialize env vars - load credentials
load_dotenv()

# initialize graph db:
graph = Neo4jGraph()

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=2000,
    )

# initialize graph transformer:
llm_transformer = LLMGraphTransformer(
    llm=llm,
    # alternative options: (these are just examples)
    # allowed_nodes=["Person", "Country", "Organization"],
    # allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)

Read reviews, prepare reviews:

In [2]:
def load_policies(
    file: str = "../data/clean/policy_chunks.csv",
    start_index: int = 0,
    limit: int = 100,
):
    # load dataframe:
    df = pd.read_csv(file)[start_index:start_index + limit]
    print(f"Loaded policies: {df.shape}")
    df["Content"] = df.apply(lambda row: f"{row['Header 1']}\n{row['Header 2']}\n{row['Concat']}".replace("\n\n", "\n").replace("\n\n", "\n"), axis=1)
    return df

Total policies: 440

Transform reviews to nodes and relationships:

In [3]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def convert_to_graph_documents(df: pd.DataFrame, page_content_column: str = "Content", llm_transformer: LLMGraphTransformer = None):
    # graph_documents = llm_transformer.convert_to_graph_documents(documents)
    loader = DataFrameLoader(df, page_content_column=page_content_column)
    documents = loader.load()
    # return  llm_transformer.convert_to_graph_documents(documents)
    graph_documents = []
    for doc in documents:
        try:
            # fireworks doesnt always return valid json
            graph_documents += llm_transformer.convert_to_graph_documents([doc])
        except:
            graph_documents += [None]
    return graph_documents

See example graphs:

In [4]:
# print(f"Nodes: {graph_documents[0].nodes}")
# print(f"Relationships: {graph_documents[0].relationships}")

Store dataframe with graph temporarily:

In [5]:
def store_graph_locally(
    df: pd.DataFrame,
    docs: list,
    path: str,
):
    df["graph"] = [gd.json() if gd is not None else None for gd in docs]
    df.to_csv(path, index=False)

Storing to graph database:

In [6]:
# graph.add_graph_documents(graph_documents)

Iteratively load these in chunks because, well, there's... a lot.

In [7]:
import time

# earlier:
# combination of fireworks and others

# together.ai: mixtral 
#start_index = 300
#limit = 500
#chunk_size = 10

start_index = 190
limit = 10000
chunk_size = 10


def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


graph_docs = []

policies_df = load_policies(start_index=start_index, limit=limit)
for idx, chunk_df in enumerate(chunker(policies_df, chunk_size)):
    idx_start = start_index + (idx * chunk_size)
    idx_end = start_index + ((idx + 1) * chunk_size)
    print(f"chunk {idx}: {idx_start} -> {idx_end}. {chunk_df.shape}")

    chunk_graph_docs = convert_to_graph_documents(chunk_df, llm_transformer=llm_transformer)
    graph_docs += chunk_graph_docs
    print(f"chunk {idx}: converted")

    store_graph_locally(chunk_df, chunk_graph_docs, f"../data/transformed/policies/policies-graph-{idx_start}-{idx_end}.csv")
    print(f"chunk {idx}: saved to csv")

    chunk_graph_docs = list(filter(None, chunk_graph_docs))
    graph.add_graph_documents(chunk_graph_docs)
    print(f"chunk {idx}: saved to neo4j")
    print(f"move index to: {idx_end}")

    time.sleep(1)

Loaded policies: (250, 4)
chunk 0: 190 -> 200. (10, 5)
chunk 0: converted
chunk 0: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 0: saved to neo4j
move index to: 200
chunk 1: 200 -> 210. (10, 5)
chunk 1: converted
chunk 1: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 1: saved to neo4j
move index to: 210
chunk 2: 210 -> 220. (10, 5)
chunk 2: converted
chunk 2: saved to csv


...

chunk 22: saved to neo4j
move index to: 420
chunk 23: 420 -> 430. (10, 5)
chunk 23: converted
chunk 23: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 23: saved to neo4j
move index to: 430
chunk 24: 430 -> 440. (10, 5)
chunk 24: converted
chunk 24: saved to csv


/var/folders/x2/3tdlb6px40qfq870vwx5y6080000gn/T/ipykernel_36981/255753804.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["graph"] = [gd.json() if gd is not None else None for gd in docs]


chunk 24: saved to neo4j
move index to: 440
